# 16.2 vLLM与Open WebUI集成部署开源大语言模型
- **目录**
  - 16.2.1 vLLM与Open WebUI简介
    - 16.2.1.1 vLLM简介
    - 16.2.1.2 Open WebUI简介
    - 16.2.1.3 vLLM与Open WebUI集成方案
  - 16.2.2 vLLM与Open WebUI部署LLM过程

- 用于开源大语言模型部署和应用的程序和工具有很多，比如LangChain(包括Web LangChain和LangServe等), FastChat, vLLM, LLMStack, llama.cpp, Gradio, Ollama, Open WebUI等。
- 本节将以vLLM和Open WebUI分别作为后端和前端框架工具，介绍开源大语言模型的自托管部署应用。

## 16.2.1 vLLM与Open WebUI简介
vLLM与Open WebUI集成是部署生产级大语言模型系统的常用配置，本节将以此两种框架在英伟达3090ti GPU上部署应用DeepSeek-R1-0528-Qwen3-8B蒸馏模型。


### 16.2.1.1 vLLM简介
- **vLLM（Vectorized Large Language Model Inference）** 是加州大学伯克利分校团队开发的开源框架，专为大语言模型（LLM）的**高效推理**设计。
  - 官方文档点击[此处](https://docs.vllm.ai/en/stable/index.html)。
  - 代码仓库点击[此处](https://github.com/vllm-project/vllm)。
- vLLM的核心目标是通过优化内存管理和计算流程，显著提升模型的吞吐量（Throughput）和响应速度，尤其适用于动态负载场景（如多用户并发请求）。
- 技术原理与核心创新：
  - （1）PagedAttention内存管理  
     - 问题背景：传统LLM推理因KV Cache占用大量连续显存，导致显存碎片化和利用率低。
     - 解决方案：借鉴操作系统分页思想，将KV Cache分割为固定大小的“块”，按需动态分配，显存利用率提升 3-4倍。
     - 效果：支持更长上下文（如32k tokens）的模型推理，且可灵活扩展。
  - （2）连续批处理（Continuous Batching）  
    - 动态调度：实时合并不同用户请求的输入序列，自动填充GPU算力空闲区域。
    - 优势：相比静态批处理，吞吐量提升 5-10倍，尤其适合实时交互场景（如ChatGPT式服务）。
  - （3）量化与分布式推理  
    - 支持AWQ(Activation-aware Weight Quantization, 激活感知权重量化)、GPTQ(GPT Quantization, GPT量化)等量化技术，降低模型显存占用。
    - 通过Tensor Parallelism和模型分片实现多卡部署，扩展至千亿参数模型。
- vLLM的特征：
  - 顶尖的推理吞吐量，vLLM 通过内存优化和高效计算，实现比传统方案（如 HuggingFace Transformers）高2-4倍的吞吐量，尤其适合高并发场景。
  - Continuous Batching（连续批处理），动态合并不同请求的输入，提高 GPU 利用率。
  - 与流行的HuggingFace模型无缝集成，直接加载 HuggingFace 模型，无需转换。
  - 通过包括并行采样、束搜索等在内的多种解码算法实现高吞吐量服务。
  - 支持张量并行和流水线并行的分布式推理。
  - 流式输出，每生成一个token或一小块文本（如 5-10 tokens）立即返回，用户可实时看到渐进式结果（如每秒看到新词）。
  - 兼容 OpenAI 的API服务器。
  - 支持NVIDIA GPU、AMD CPU和GPU、Intel CPU、Gaudi加速器和 GPU、IBM Power CPU、TPU以及AWS Trainium和Inferentia加速器。
  - 前缀缓存支持，缓存公共提示词前缀（如系统指令），加速重复请求。
  - Multi-lora支持，同时加载多个LoRA微调模型，动态切换。
- vLLM 的实践优势 
| 对比项          | vLLM                          | HuggingFace Transformers      | DeepSpeed Inference |
|-----------------|-------------------------------|--------------------------------|---------------------|
| 吞吐量       | ⭐⭐⭐⭐（最高）                 | ⭐⭐                           | ⭐⭐⭐               |
| 延迟控制     | 动态优化请求优先级             | 静态批处理，延迟波动大          | 中等                |
| 硬件适配     | 专为NVIDIA GPU优化，支持多节点 | 通用GPU/CPU                   | 需定制配置          |
| 部署复杂度   | 低（API服务器一键启动）        | 中（需手动优化）               | 高（依赖DS配置）    |
- 典型应用场景 
  - （1）在线服务与API  
    - 支持高并发请求的聊天机器人、内容生成平台（如客服系统、营销文案生成）。
  - （2）研究实验加速  
    - 快速迭代模型变体（如微调后的Llama-3、Mixtral），减少实验周期。
  - （3）边缘计算与云服务  
    - 结合量化技术，在边缘设备（如NVIDIA Jetson）部署70B以下模型。
  - （4）多模态扩展  
    - 通过适配器（Adapter）支持视觉-语言模型（如LLaVA）的混合推理。
- 挑战与局限性 
  - 硬件依赖：目前主要针对NVIDIA GPU（CUDA），对AMD/国产芯片支持有限。
  - 调试复杂性：动态内存管理可能导致非常规错误的排查难度增加。
  - 生态兼容性：与部分定制化模型结构（如MoE特定实现）存在兼容性问题。

- vLLM简单示例

In [ ]:
from vllm import LLM
prompts = ["Hello, my name is", "The capital of France is"]  # Sample prompts.
llm = LLM(model="../huggingface/DeepSeek-R1-0528-Qwen3-8B")  # Create an LLM.
outputs = llm.generate(prompts)  # Generate texts from the prompts.

- 使用 vLLM 进行在线服务，可以在linux等系统的终端通过以下方式启动一个与 OpenAI API 兼容的服务器：
  - 第一种使用vLLM Python API方式启动部署vLLM（可通过运行参数进行深度定制）：
    ```bash
    python -m vllm.entrypoints.openai.api_server --model lmsys/vicuna-7b-v1.3
    ```    
  - 第二种使用vLLM CLI方式启动部署vLLM（可以通过启动参数进行深度定制）：
    ```bash
    vllm serve
    ```
- 然后使用与 OpenAI API 相同的格式查询服务器：
  ```bash
    curl http://localhost:8000/v1/completions \
    -H "Content-Type: application/json" \
    -d '{
        "model": "lmsys/vicuna-7b-v1.3",
        "prompt": "San Francisco is a",
        "max_tokens": 7,
        "temperature": 0
    }'
  ```
  

### 16.2.1.2 Open WebUI简介
- Open WebUI是一个功能丰富且用户友好的**开源自托管AI平台**，支持完全离线部署，能够与多种大型语言模型后端无缝集成，包括Ollama和OpenAI兼容API。
  - 该平台内置了检索增强生成(RAG)推理引擎，提供了强大的AI部署解决方案。
  - Open WebUI的代码仓库在[此处](https://github.com/open-webui/open-webui)。
- 在技术架构上，Open WebUI采用模块化设计。
  - 前端基于Vue.js/TypeScript。
  - 后端使用Python FastAPI，一种基于 Python 的 现代 Web 框架。
  - 支持通过Docker或Kubernetes快速部署，并提供CPU和GPU两个版本镜像。
- 平台具有完善的用户权限管理系统。
  - 支持基于角色的访问控制(RBAC，Role-Based Access Control)。
  - 管理员可以创建多级用户角色，实现精细化的权限分配，确保只有授权用户才能访问特定模型和功能。
- Open WebUI提供了全面的模型管理功能。
  - 用户可以通过Web界面直接创建和管理Ollama模型，支持自定义角色和代理设置。
  - 平台支持多模型并行对话，允许用户同时与多个模型交互，充分利用不同模型的特点。
- 在交互体验方面：
  - Open WebUI采用响应式设计，完美适配PC、笔记本和移动设备，并支持渐进式Web应用(PWA，Progressive Web App)技术，实现类原生应用的移动端体验。
  - 平台内置完整的Markdown和LaTeX支持，能够渲染复杂的数学公式和代码块，同时集成了语音视频通话功能，为用户提供更丰富的交互方式。
- 在知识处理和扩展能力方面：
  - Open WebUI深度整合了检索增强生成(RAG)技术，用户可以直接上传文档或建立文档库，通过#命令调用相关内容。
  - 平台还支持联网搜索，能够接入SearXNG、Google PSE等多种搜索引擎，并将搜索结果实时注入对话上下文。
  - Open WebUI具备网页内容抓取能力，用户只需输入#加URL即可将网页内容导入对话。
  - 在多媒体支持上，平台集成了ComfyUI等本地图像生成工具以及DALL-E等云端服务，为用户提供图文并茂的交互体验。
    - ComfyUI 是一个基于节点式工作流（Node-based）的 AI 图像生成与处理工具。
    - 专注于支持 Stable Diffusion 等扩散模型的精细化控制。
    - 通过可视化编程的方式，允许用户通过连接不同的功能模块（节点）来构建复杂的图像生成流程。
    - ComfyUI的代码仓库在[此处](https://github.com/comfyanonymous/ComfyUI)。
- Open WebUI具有高度的可扩展性，支持多语言国际化(i18n)界面，并提供了完善的插件开发框架。
- 开发者可以通过Pipelines插件系统集成自定义Python函数，实现函数调用、使用量监控、内容过滤等高级功能。
- 平台还支持与Langfuse等监控工具集成，满足企业级应用的需求。


- Open WebUI的总体架构如下图所示：

```mermaid
graph TB
  A[用户界面] --> B{后端服务}
  B --> C[Ollama/vLLM/OpenAI API]
  B --> D[向量数据库]
  B --> E[插件系统]
  C --> F[多模型并行]
  D --> G[RAG增强]
  E --> H[Python函数/第三方服务]
```
<center >图16.2.1 Open WebUI总体架构图</center>

- Open WebUI的两种部署方式：
  - **Open WebUI的Docker部署方式**是一种高度标准化、适合生产环境的方案，通过容器化技术实现快速部署和资源隔离。
    - 用户只需拉取**预构建**的Docker镜像，通过简单的命令即可启动服务，无需手动安装依赖或配置复杂环境。
      ```bash
      docker pull ghcr.io/open-webui/open-webui:main 
      docker run -d \
       --name open-webui \
       -p 3000:8080 \                          # 将容器内8080端口映射到宿主机3000
       -e OLLAMA_API_BASE_URL=http://host.docker.internal:11434 \  # 默认Ollama地址
       -v open-webui-data:/app/backend/data \   # 持久化数据卷
       --gpus all \                            # 启用GPU支持（若需AI功能）
       ghcr.io/open-webui/open-webui:main
       ```
    - 关键优势在于其**开箱即用**的特性——通过环境变量（如`OPENAI_API_BASE_URL`）可灵活对接不同后端模型服务（如vLLM或Ollama），数据卷挂载确保聊天记录持久化，`--gpus all`参数可以充分利用所有可用的GPU支持加速推理。
    - 此方式特别适合需要稳定运行、多用户协作或云服务器部署的场景，但调试时需依赖容器日志分析。  
  - **Open WebUI的源码启动方式（`open-webui serve`）** 则面向开发者，提供全栈代码控制权。
    - 通过克隆仓库并手动安装前后端依赖，用户可直接修改界面逻辑或API接口，实时调试变更。
    - 启动时通过参数（如`--openai-api-url`）指定模型服务地址，支持动态调整端口和认证设置。
      ```bash
      git clone https://github.com/open-webui/open-webui.git
      open-webui serve
      ```
    - 这种方式牺牲了部署便捷性，但赋予开发者深度定制能力，例如集成企业级身份验证或优化UI交互，适合二次开发、功能测试或贡献开源代码的场景。
    - 其劣势在于依赖宿主机环境，且需自行处理服务监控与数据备份。

### 16.2.1.3 vLLM与Open WebUI集成方案
- **核心目标**
  - **高性能推理**：利用 vLLM 的 PagedAttention 和连续批处理技术，提升大模型吞吐量。
  - **统一接口**：通过 Open WebUI 的标准化界面操作 vLLM 托管的模型。
- **技术互补性**
| 组件          | 核心贡献                                      | 集成价值                                      |
|---------------|---------------------------------------------|---------------------------------------------|
| **vLLM**      | 提供低延迟、高吞吐的 LLM 推理引擎              | 解决 Open WebUI 本地推理的性能瓶颈            |
| **Open WebUI** | 提供用户友好的 Web 界面和会话管理功能          | 为 vLLM 添加生产级交互和管理能力              |


- **系统架构**

```mermaid
sequenceDiagram
    participant User as 用户浏览器
    participant WebUI as Open WebUI (FastAPI)
    participant vLLM as vLLM 推理引擎
    participant GPU as GPU 资源

    User->>WebUI: 发起请求 (HTTP/WebSocket)
    WebUI->>vLLM: 转发请求 (OpenAI兼容API)
    vLLM->>GPU: 执行推理 (PagedAttention)
    GPU->>vLLM: 返回生成结果
    vLLM->>WebUI: 返回标准化响应
    WebUI->>User: 渲染结果 (流式/非流式)
```
<center >图16.2.2 vLLM与Open WebUI集成的系统架构</center>

- **vLLM与Open WebUI的集成方式**
  - **方案 1：vLLM CLI + Open WebUI Docker**：
    - 使用 `vllm serve` 快速启动模型服务，并通过 Docker 部署 Open WebUI。
    - 需在 Open WebUI 容器中设置环境变量，将其请求转发至 vLLM 的端口。
    - 此组合兼顾了易用性和稳定性，适合大多数生产场景。
  - **方案 2：vLLM Python API + Open WebUI 源码**：
    - 通过 Python 模块启动 vLLM 以支持高级参数调优，同时以源码模式`open-webui serve`运行 Open WebUI 实现深度定制。
    - 需手动确保两者网络互通（如本地回环地址或局域网 IP）。
    - 此组合灵活性最高，但维护成本较大。
  - **方案 3：vLLM CLI + Open WebUI 源码**：
    -  使用 `vllm serve` 快速启动模型服务，同时以源码模式`open-webui serve`运行 Open WebUI 实现深度定制。
    -  此组合简单易行，本节将使用此方式进行集成部署。

## 16.2.2 vLLM与Open WebUI部署LLM过程
具体部署步骤如下：
- **（1）平台和软件版本**
  - GPU显卡：英伟达GeForce RTX 3090ti，24G显存
  - 英伟达GPU驱动程序版本：550.144.03，CUDA版本：12.4
  - 操作系统： Ubuntu 22.04.1 LTS
  - Python：3.11
  - vLLM：0.9.1
  - Open WebUI：0.6.15
- **（2）安装vLLM和Open WebUI**
  - 分别使用`pip3 install vllm`和`pip3 install open-webui`安装框架的Python包。
  - 两个包都比较大，安装需要花一点时间，具体时长取决于网速。

- **（3）下载模型**
  - 此处仍是使用16.1节已下载的模型DeepSeek-R1-0528-Qwen3-8B。
  - 存放在某个目录下，比如16.1节中存放在深度学习课程各章代码文件夹同级目录下的huggingface目录，如下图所示：
  <img src='../img/16_3_2.png'> 

- **（4）启动vLLM**
  - 进入与huggingface同级的路径下，即~/python/deeplearning，运行如下命令，启动vLLM:
    ```bash
       vllm serve huggingface/DeepSeek-R1-0528-Qwen3-8B --port 5000 --trust-remote-code --max-model-len 8192 --gpu-memory-utilization 0.85 --dtype float16 --enforce-eager --disable-log-stats --block-size 16 --max-num-batched-tokens 8192 --max-num-seqs 128
    ```
  - 启动参数说明：
    - **`huggingface/DeepSeek-R1-0528-Qwen3-8B`** ：指定加载的模型名称或路径（Hugging Face 模型库或本地目录），此处为 `DeepSeek-R1-0528-Qwen3-8B` 的 8B 参数量版本。注意：本次部署使用已下载模型，本命令使用的模型所在的相对路径，当然也可以使用绝对路径。即模型在运行`vllm serve`命令的同级路径下的`huggingface`路径下。
    - **`--port 5000`**：设置服务监听端口为 `5000`（默认 8000），需确保端口未被占用。注意：后面配置Open WebUI的`OPENAI_API_BASE_URL`环境变量时要和此端口保持一致。
    - **`--trust-remote-code`** ：允许加载模型中的自定义代码（如非标准架构的模型需启用此选项）。
    - **`--gpu-memory-utilization 0.85`**：GPU 显存利用率上限设为 85%，避免显存耗尽导致 OOM（根据 GPU 总显存动态调整）。
    - **`--dtype float16`**：指定模型计算精度为 `float16`（半精度），平衡精度与显存占用（可选 `bfloat16` 或 `float32`）。
    - **`--enforce-eager`**：禁用计算图优化（PyTorch 的 `eager` 模式），兼容性更强但可能降低性能（某些自定义模型需启用）。
    - **`--max-model-len 8192`**：模型支持的最大上下文长度（token 数），此处设为 8192（需模型本身支持长上下文）。
    - **`--max-num-batched-tokens 8192`**：单批次处理的 token 总数上限，影响吞吐量（值越大，并发能力越强，但显存占用越高）。
    - **`--max-num-seqs 128`**：同时处理的最大请求数（队列长度），高并发场景需调高此值（但受显存限制）。
    - **`--block-size 16`**：PagedAttention 的块大小（token 数），影响显存碎片管理（默认 16，通常无需修改）。
    - **`--disable-log-stats`**：禁用推理统计日志（如延迟、吞吐量），减少日志输出量（生产环境建议启用以监控性能）。
  - 启动界面如下图所示：
    <img src='../img/16_3_3.png' /> 
    <img src='../img/16_3_4.png' />
    <img src='../img/16_3_5.png' />
    <img src='../img/16_3_6.png' />
  - 测试vLLM是否成功启动，在浏览器地址栏输入`http://127.0.0.1:5000/v1/models`或`http://localhost:5000/v1/models`，如果显示信息如下图所示，表明vLLM启动成功：
    <img src='../img/16_3_7.png' />

- **（5）启动Open WebUI**
  - 设置环境变量。
    - 在终端输入如下命令：  
    ```bash
    export ENABLE_OLLAMA_API=False  # 禁止后台尝试连接 Ollama
    export OPENAI_API_BASE_URL=http://127.0.0.1:5000/v1 # 后台使用vLLM的服务API接口
    export DEFAULT_MODELS="huggingface/DeepSeek-R1-0528-Qwen3-8B" # 该值必须与vLLM启动时加载的模型名完全一致
    export HF_ENDPOINT=https://hf-mirror.com # 设置huggingface镜像，否则国内使用Open WebUI将会存在极大障碍
    ```
    - 可通过`echo $OPENAI_API_BASE_URL`等查看环境变量的值。
    - 环境变量设置界面如下：
      <img src='../img/16_3_8.png' />
  - 运行`open-webui serve`命令，如下图所示：
    <img src='../img/16_3_9.png' />
    <img src='../img/16_3_10.png' />
    <img src='../img/16_3_11.png' />

- **（6）应用测试**
  - 在部署服务器的浏览器地址栏输入`http://127.0.0.1:8080`或远程访问`http://192.168.11.203:8080/`。
  - 初次访问模型页面，Open WebUI会显示一个注册窗口，输入`Username`，`Email`和`Password`。
  - 注册成功之后，系统自动跳转到登录界面，用户即可登录系统。
    <img src='../img/16_3_13.png' />
  - 进入主页之后即可看到类似如下界面：
    <img src='../img/16_3_15.png' />
  - 输入提示，提交之后的界面如下图：
    <img src='../img/16_3_12.png' />
  - 其他功能：
    <img src='../img/16_3_14.png' />
    <img src='../img/16_3_16.png' /> 

## 小结
- 介绍vLLM和Open OpenUI大语言模型部署工具。
- 集成应用vLLM和Open OpenUI进行大语言模型的实际部署和应用测试。